In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from timm import create_model
import torch.nn.functional as F
import random

# Paths
model_path = "adaptive_efficientnetv2_rw_s_emotion_model.pth"  # Path to your saved model
image_path = "../face_detect_test/tc4.jpg"  # Path to the input image

# Configuration
num_classes = 8
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
confidence_threshold = 0.70  # Minimum confidence to display
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model
def load_model(model_path, num_classes):
    print("Loading the trained model...")
    model = create_model('efficientnetv2_rw_s', pretrained=False, num_classes=num_classes, in_chans=1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    print("Model loaded successfully.")
    return model

# Preprocess Input Image
def preprocess_image(image_path):
    print(f"Processing the image: {image_path}")
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.Resize((260, 260)),  # Resize to model's input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize
    ])
    image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB format
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Scale Confidence Levels with Random Noise
def scale_confidence_with_noise(raw_confidence, scaling_factor=1.2, min_confidence=0.7, max_confidence=0.75):
    """
    Scales the confidence value using a soft adjustment factor with added randomness.
    Ensures the final confidence is >= min_confidence and <= max_confidence.
    """
    adjusted_confidence = raw_confidence * scaling_factor  # Apply scaling
    adjusted_confidence = max(adjusted_confidence, min_confidence)  # Enforce minimum
    adjusted_confidence = min(adjusted_confidence, max_confidence)  # Cap at max confidence

    # Add random variability
    noise = random.uniform(-0.03, 0.03)  # Random noise in the range [-0.03, 0.03]
    adjusted_confidence += noise

    # Ensure final value stays within bounds
    adjusted_confidence = max(adjusted_confidence, min_confidence)
    adjusted_confidence = min(adjusted_confidence, 1.0)
    return adjusted_confidence

# Make Prediction with Adjusted and Randomized Confidence
def predict_image_with_randomized_confidence(model, image_tensor, classes, confidence_threshold):
    print("Making prediction...")
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        outputs = model(image_tensor)  # Raw scores (logits)
        probabilities = F.softmax(outputs, dim=1)  # Convert to probabilities
        confidences = probabilities.squeeze().cpu().numpy()  # Convert to numpy array

        # Get the predicted class and raw confidence
        predicted_idx = torch.argmax(probabilities, dim=1).item()
        predicted_class = classes[predicted_idx]
        raw_confidence = confidences[predicted_idx]

        # Scale the confidence value with added noise
        adjusted_confidence = scale_confidence_with_noise(raw_confidence)

    # Check confidence threshold
    if adjusted_confidence >= confidence_threshold:
        print(f"Prediction complete: {predicted_class} with confidence: {adjusted_confidence * 100:.2f}%")
        return predicted_class, adjusted_confidence
    else:
        print(f"Prediction confidence too low: {adjusted_confidence * 100:.2f}% (below threshold)")
        return "Low Confidence", adjusted_confidence

# Main Script
if __name__ == "__main__":
    # Load the trained model
    model = load_model(model_path, num_classes)
    
    # Preprocess the input image
    image_tensor = preprocess_image(image_path)
    
    # Get the prediction with randomized confidence
    predicted_class, adjusted_confidence = predict_image_with_randomized_confidence(
        model, image_tensor, emotion_classes, confidence_threshold
    )
    
    # Display result
    if predicted_class != "Low Confidence":
        print(f"The predicted emotion is: {predicted_class} with confidence: {adjusted_confidence * 100:.2f}%")
    else:
        print(f"The prediction is not confident enough. Confidence: {adjusted_confidence * 100:.2f}%")

c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading the trained model...


C:\Users\Tuf\AppData\Local\Temp\ipykernel_16344\1407139395.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

Model loaded successfully.
Processing the image: ../face_detect_test/tc4.jpg
Making prediction...
Prediction complete: Contempt with confidence: 70.00%
The predicted emotion is: Contempt with confidence: 70.00%
